# IPFP/Sinkorn

In [9]:
import numpy as np
import gurobipy as grb
import os
import pandas as pd
import time
import scipy.sparse as spr
#import rpy2.robjects as robjects

In [199]:
nbX = 5
nbY = 3
tol = 1e-9
maxite = 1e+06

In [200]:
thepath = os.path.join(os.getcwd(),'..')
data_X = pd.read_csv(os.path.join(thepath, "data_mec_optim/marriage_personality-traits/Xvals.csv"))
data_Y = pd.read_csv(os.path.join(thepath, "data_mec_optim/marriage_personality-traits/Yvals.csv"))
affdf = pd.read_csv(os.path.join(thepath,"data_mec_optim/marriage_personality-traits/affinitymatrix.csv"))
nbcar = 10
affmat = affdf.iloc[0:nbcar,1:nbcar+1].values
sdX = data_X.std().values
sdY = data_Y.std().values
mX = data_X.mean().values
mY = data_Y.mean().values

Xvals = ((data_X-mX)/sdX).values
Yvals = ((data_Y-mY)/sdY).values
nobs = Xvals.shape[0]
Phi = (Xvals @ affmat @ Yvals.T)[:nbX,:nbY]
obj = Phi.flatten()
    
p = np.repeat(1/nbX, nbX)
q = np.repeat(1/nbY, nbY)

In [201]:
nrow = min(8, nbX)
ncol = min(8, nbY)

In Gurobi

In [202]:
ptm = time.time()
A1 = spr.kron(np.ones((1, nbY)), spr.identity(nbX))
A2 = spr.kron(spr.identity(nbY), np.ones((1, nbX)))
A = spr.vstack([A1, A2])
obj = Phi.flatten(order = 'F') # flatten order is important
rhs = np.hstack([p,q])
    
m = grb.Model('marriage')
x = m.addMVar(len(obj), name='couple')
m.setObjective(obj @ x, grb.GRB.MAXIMIZE)
m.addConstr(A @ x == rhs, name="Constr")
m.setParam( 'OutputFlag', False ) #quiet output
m.optimize()
diff = time.time() - ptm
print('Time elapsed (Gurobi) = ', diff, 's.')
if m.status == grb.GRB.Status.OPTIMAL:
    val_gurobi = m.objval
    x = m.getAttr('x')
    x = np.array(x).reshape([nbX, nbY])
    pi = m.getAttr('pi')
    u_gurobi = pi[:nbX]
    v_gurobi = pi[nbX:nbX + nbY]
    print("Value of the problem (Gurobi) = ", val_gurobi)
    print(np.subtract(u_gurobi[:nrow], u_gurobi[nrow - 1]))
    print(np.add(v_gurobi[:ncol], u_gurobi[nrow - 1]))
    print('*************************')

Time elapsed (Gurobi) =  0.00594639778137207 s.
Value of the problem (Gurobi) =  0.41095324822187485
[-0.63237262 -0.57124993  1.634949   -1.24417523  0.        ]
[0.62598494 0.61304671 0.48153736]
*************************


Using linear IPFP

Quick note here. Numpy doesn't exactly do what one would expect when broadcasting 1d arrays to 2 arrays (for example when adding vectors and matrices). Be really careful here because things can get messy. I like to change all vectors (1d objects) into matrices with a single column to be sure that broadcasting is done right.

In [203]:
def two_d(X):
    return np.reshape(X,(X.size, 1))

In [222]:
ptm = time.time()
ite = 0
sigma = 0.1

K = np.exp(Phi/sigma)
B = two_d(np.repeat(1, nbY))
error = tol + 1
    
while error > tol and ite < maxite:
    A = two_d(p/(K @ B).flatten(order='F'))
    KA = (A.T @ K)
    error = np.max(abs(np.multiply(KA,B.flatten()/q)-1))
    B = (q / KA).T
    ite = ite + 1
        
u = - sigma * np.log(A)
v = - sigma * np.log(B)
pi = (K * A) * np.repeat(B, 5, axis = 1).T
val = np.sum(pi * Phi) - sigma * np.sum(pi * np.log(pi))
end = time.time() - ptm
if ite >= maxite:
    print('Maximum number of iteations reached in IPFP1.')
else:
    print('IPFP1 converged in ', ite, ' steps and ', end, 's.')
    print('Value of the problem (IPFP1) = ', val)
    print('Sum(pi*Phi) (IPFP1) = ', np.sum(np.multiply(pi,Phi)))
    print('*************************')

IPFP1 converged in  89  steps and  0.0060002803802490234 s.
Value of the problem (IPFP1) =  0.6045556509904391
Sum(pi*Phi) (IPFP1) =  0.4001284575694894
*************************


Log IPFP

In [275]:
sigma = 0.01
ptm = time.time()
ite = 0
v = np.repeat(0, nbY)
mu = - sigma * np.log(p)
nu = - sigma * np.log(q)
error = tol + 1
while error > tol and ite < maxite:
    u = mu + sigma * np.log(np.sum(np.exp((Phi - np.repeat(two_d(v), nbX, axis = 1).T)/sigma), axis=1))
    KA = np.sum(np.exp((Phi - two_d(u)) / sigma), axis=0)
    error = np.max(np.abs(KA * np.exp(-v / sigma) / q - 1))
    v = nu + sigma * np.log(KA)
    ite = ite + 1
pi = np.exp((Phi - two_d(u) - np.repeat(two_d(v), nbX, axis = 1).T) / sigma)
val = np.sum(pi * Phi) - sigma * np.sum(pi * np.log(pi_bis))
end = time.time() - ptm

if ite >= maxite:
    print('Maximum number of iteations reached in IPFP1.')
else:
    print('IPFP1_logs converged in ', ite, ' steps and ', end, 's.')
    print('Value of the problem (IPFP1_logs) = ', val)
    print('Sum(pi*Phi) (IPFP1_logs) = ', np.sum(pi *Phi))
    print('*************************')

IPFP1_logs converged in  156  steps and  0.01238870620727539 s.
Value of the problem (IPFP1_logs) =  0.43031337771843775
Sum(pi*Phi) (IPFP1_logs) =  0.4109531251395014
*************************


Both procedures above will break down when $\sigma$ is small, e.g. $\sigma=0.001$ (Try!). However if we modify the second procedure using the log-sum-exp trick, things work again:

In [346]:
sigma = 0.001
ptm = time.time()
ite = 0
v = np.repeat(0, nbY)
mu = - sigma * np.log(p)
nu = - sigma * np.log(q)
error = tol + 1
uprec = np.NINF
while error > tol and ite < maxite:
    vstar = np.max(Phi.T - two_d(v), axis = 0)
    u = mu + vstar + sigma * np.log(np.sum(np.exp((Phi - np.repeat(two_d(v), nbX, axis = 1).T - 
                                                   two_d(vstar))/sigma), axis=1))
    error = np.max(abs(u - uprec))
    uprec = u
    ustar = np.max(Phi - two_d(u), axis = 0)
    KA = np.sum(np.exp((Phi - two_d(u) - np.repeat(two_d(ustar), nbX, axis = 1).T) / sigma), axis=0)

    v = nu + ustar + sigma * np.log(KA)
    ite = ite + 1
pi = np.exp((Phi - two_d(u) - np.repeat(two_d(v), nbX, axis = 1).T) / sigma)
val = np.sum(pi * Phi) - sigma * np.sum(pi * np.log(pi))
end = time.time() - ptm

if ite >= maxite:
    print('Maximum number of iteations reached in IPFP1.')
else:
    print('IPFP1_logs converged in ', ite, ' steps and ', end, 's.')
    print('Value of the problem (IPFP1_logs) = ', val)
    print('Sum(pi*Phi) (IPFP1_logs) = ', np.sum(pi *Phi))
    print('*************************')

IPFP1_logs converged in  315  steps and  0.04296588897705078 s.
Value of the problem (IPFP1_logs) =  nan
Sum(pi*Phi) (IPFP1_logs) =  0.41095352613797476
*************************


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in multiply
